## Adding user ids and user location information to the data we got by using "Get Old Tweets 3" library

In [1]:
import tweepy
import pandas as pd
import json

#Credentials
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [60]:
# Reading the dataset; this was done to each hashtag data file separately, by changing the file that is being read in
data = pd.read_csv('parking_from_1st_OCTOBER.csv', encoding = "latin") 

In [62]:
# Some preparation for data set
data['id'] = None
data = data.rename(columns={"id": "user_id"})

In [63]:
screen_names = data['username'].unique()

In [64]:
len(screen_names) # This many unique users in the data set

6383

In [66]:
names_to_id = {}

for sname in screen_names:
    try:
        user = api.get_user(screen_name = sname) # Get user object by screen name
        uid_str = user.id_str
        uid = user.id
        location = user.location
        names_to_id[sname] = (uid_str, uid, location) # Save the integer (and string) user id and location for the current user into a dictionary
        
    except Exception as e: # For example, user can have been suspended
        names_to_id[sname] = (None, None, None)

In [70]:
data['user_id'] = data['username'].apply(lambda x: names_to_id[x][1])
data['location'] = data['username'].apply(lambda x: names_to_id[x][2])
data['user_id_str'] = data['username'].apply(lambda x: names_to_id[x][0])

In [73]:
# Saving the modified data file
data.to_csv('parking_final.csv', index=False)

In [68]:
# Saving the dictionary to JSON file
with open('parking_user_ids_locations.json', 'w') as file:
    json.dump(names_to_id, file)